In [1]:
from src.utils import get_files_from_folder, open_txt
from src.model import BigramLanguageModel
import torch
import torch.nn as nn
from torch.nn import functional as F

In [2]:
books = get_files_from_folder("books")
books_string = [open_txt(f"books/{i}") for i in books]
print(books_string[0][:500])

NOTES FROM THE UNDERGROUND[*]
A NOVEL


* The author of the diary and the diary itself are, of course,
imaginary. Nevertheless it is clear that such persons as the writer of
these notes not only may, but positively must, exist in our society,
when we consider the circumstances in the midst of which our society is
formed. I have tried to expose to the view of the public more
distinctly than is commonly done, one of the characters of the recent
past. He is one of the representatives of a generatio


In [3]:
all_books = "\n".join(books_string)
print(f"All books have a lenght of: {len(all_books)}")

vocab = sorted(set(all_books))
vocab_size = len(vocab)
print(vocab)
print(f"The vocabulary has a lenght of: {len(vocab)}")

All books have a lenght of: 7113352
['\n', ' ', '!', '"', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'À', 'Æ', 'É', 'à', 'â', 'ä', 'æ', 'ç', 'è', 'é', 'ê', 'ë', 'î', 'ï', 'ô', 'ö', 'ü', 'Œ', 'œ', '‐', '—', '‘', '’', '“', '”']
The vocabulary has a lenght of: 104


In [4]:
stoi = {c: i for i, c in enumerate(vocab)}
itos = {i: c for i, c in enumerate(vocab)}
encode = lambda x: [stoi[c] for c in x]
decode = lambda x: "".join([itos[c] for c in x])

print(encode("Dostoyevsky"))
print(decode(encode("Dostoyevsky")))

[27, 67, 71, 72, 67, 77, 57, 74, 71, 63, 77]
Dostoyevsky


In [5]:
data = torch.tensor(encode(all_books), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:10])

torch.Size([7113352]) torch.int64
tensor([37, 38, 43, 28, 42,  1, 29, 41, 38, 36])


/tmp/ipykernel_13240/3704913605.py:1: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:84.)
  data = torch.tensor(encode(all_books), dtype=torch.long)


In [6]:
train_size = int(len(data) * 0.9)
train_data = data[:train_size]
val_data = data[train_size:]

In [7]:
batch_size = 4
block_size = 8


def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i : i + block_size] for i in ix])
    y = torch.stack([data[i + 1 : i + block_size + 1] for i in ix])
    return x, y


xb, yb = get_batch("train")
print("inputs:")
print(xb.shape)
print(xb)
print("targets:")
print(yb.shape)
print(yb)

print("----")

for b in range(batch_size):  # batch dimension
    for t in range(block_size):  # time dimension
        context = xb[b, : t + 1]
        target = yb[b, t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[  1,  65,  57,   1,  59,  67,  10, 103],
        [ 61,  57,  70,  71,   1,  71,  53,  77],
        [ 66,  64,  77,   1,  55,  53,  65,  57],
        [ 61,  65,  61,  66,  59,  10,   1,  27]])
targets:
torch.Size([4, 8])
tensor([[ 65,  57,   1,  59,  67,  10, 103,   0],
        [ 57,  70,  71,   1,  71,  53,  77,  10],
        [ 64,  77,   1,  55,  53,  65,  57,   1],
        [ 65,  61,  66,  59,  10,   1,  27,  53]])
----
when input is [1] the target: 65
when input is [1, 65] the target: 57
when input is [1, 65, 57] the target: 1
when input is [1, 65, 57, 1] the target: 59
when input is [1, 65, 57, 1, 59] the target: 67
when input is [1, 65, 57, 1, 59, 67] the target: 10
when input is [1, 65, 57, 1, 59, 67, 10] the target: 103
when input is [1, 65, 57, 1, 59, 67, 10, 103] the target: 0
when input is [61] the target: 57
when input is [61, 57] the target: 70
when input is [61, 57, 70] the target: 71
when input is [61, 57, 70, 71] the target: 1
when inp

In [8]:
m = BigramLanguageModel()
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(
    decode(
        m.generate(idx=torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[
            0
        ].tolist()
    )
)

torch.Size([32, 104])
tensor(5.3548, grad_fn=<NllLossBackward0>)

G”‘éT Oâ*l”i“J5[PYX'ôK8q(6SBPuA)”UFœæI)î26X—î0ZKpaL]UF.ïï:’,"AzBVdöOHl
vkbiXNw_QHI8pQc”ë5csXKV”Gä —”


In [9]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [10]:
batch_size = 32
for steps in range(10000):  # increase number of steps for good results...
    # sample a batch of data
    xb, yb = get_batch("train")

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    if steps % 1000 == 0:
        print(f"Loss for {steps} : {loss.item()}")
print(f"Loss for {steps} : {loss.item()}")

Loss for 0 : 5.358506679534912
Loss for 1000 : 4.173586845397949
Loss for 2000 : 3.422147512435913
Loss for 3000 : 2.9323503971099854
Loss for 4000 : 2.7229998111724854
Loss for 5000 : 2.5551624298095703
Loss for 6000 : 2.4903650283813477
Loss for 7000 : 2.6020524501800537
Loss for 8000 : 2.4912445545196533
Loss for 9000 : 2.521933078765869
Loss for 9999 : 2.5369133949279785


In [11]:
print(
    decode(
        m.generate(idx=torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[
            0
        ].tolist()
    )
)


besenif rnZâŒ, ind ainïPheme ma bleay weld, As the orin bybooie r teemid _C39stherchre s, onbllyove 


In [12]:
torch.save(
    {
        "state_dict": m.state_dict(),
    },
    "model_store/fyodor.pt",
)

In [13]:
nlp = BigramLanguageModel()
nlp.token_embedding_table.weight

Parameter containing:
tensor([[ 0.2936, -0.1864,  1.3101,  ..., -0.1238,  1.3173, -1.5328],
        [ 0.1537, -0.3418,  0.2981,  ..., -0.3151,  1.1897, -0.5324],
        [ 1.2973,  1.6508, -0.1868,  ...,  1.7682,  0.5678,  0.7637],
        ...,
        [-0.6019,  0.0391, -0.1982,  ...,  0.1669,  0.0601,  0.0303],
        [-0.9081,  1.4839,  1.3801,  ..., -0.7037,  0.1223,  1.0521],
        [-0.8782, -0.2263, -0.8543,  ...,  0.2864, -1.9203,  0.5960]],
       requires_grad=True)

In [14]:
load = torch.load("model_store/fyodor.pt")
nlp.load_state_dict(load["state_dict"])

<All keys matched successfully>

In [15]:
nlp.token_embedding_table.weight

Parameter containing:
tensor([[ 2.5532, -1.2180, -5.8325,  ..., -2.2854,  1.7837, -3.5798],
        [-4.3733, -1.7908, -4.5328,  ..., -5.0469, -0.8697, -4.1664],
        [-0.3309,  1.7659, -4.6978,  ..., -0.9283, -3.4268,  1.7296],
        ...,
        [-0.8234,  1.0634, -2.4750,  ..., -5.0643, -4.4327, -2.3750],
        [-4.0360, -1.3939, -5.0085,  ..., -2.4442, -4.0667, -3.8627],
        [ 2.5001,  2.5350, -4.8955,  ..., -4.9714, -4.4735, -5.6828]],
       requires_grad=True)

In [16]:
print(
    decode(
        nlp.generate(idx=torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[
            0
        ].tolist()
    )
)


a athof w

hemeneo t ime
atotermursid
n or tilas; lis donoour. has Be g hohole I me asy waneridrâlin
